<a href="https://colab.research.google.com/github/trungduyen0220/text-classification-assignment/blob/master/Assignment1_NguyenThiMyDuyen_MSE13028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -f train.csv test.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv

--2020-09-06 08:21:03--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10400749 (9.9M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   9.92M  25.8MB/s    in 0.4s    

2020-09-06 08:21:03 (25.8 MB/s) - ‘train.csv’ saved [10400749/10400749]

--2020-09-06 08:21:03--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1133794 (1.1M) [text/plain]
Saving

In [2]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
train_df.head()

,id,title,abstract,category
0,1901.05003,Quantum Teleportation-Inspired Algorithm for S...,We show that low-depth random quantum circui...,quant-ph
1,1903.08905,RAP-Net: Recurrent Attention Pooling Networks ...,The response selection has been an emerging ...,cs
2,1907.11533,Combinatorial protein-protein interactions on ...,Scaffold proteins organize cellular processe...,q-bio
3,1711.07582,CVXR: An R Package for Disciplined Convex Opti...,CVXR is an R package that provides an object...,stat
4,1910.05833,Analytical Solution of (2+1) Dimensional Dirac...,"In this article, we studied the system of (2...",quant-ph
